In [1]:


import pandas as pd
import pickle
import pprint
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import np
import matplotlib as plt
import os

import re
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt



In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/ubd/anaconda/envs/topic_modelling/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:

snsf_ge_abs_df = pd.read_csv("../data/P3_GrantExport_with_abstracts.csv", sep=";")

In [4]:
snsf_ge_abs_df.keys()

Index(['Project Number', 'Project Number String', 'Project Title',
       'Project Title English', 'Responsible Applicant', 'Funding Instrument',
       'Funding Instrument Hierarchy', 'Institution', 'Institution Country',
       'University', 'Discipline Number', 'Discipline Name',
       'Discipline Name Hierarchy', 'All disciplines', 'Start Date',
       'End Date', 'Approved Amount', 'Keywords', 'Abstract',
       'Lay Summary Lead (English)', 'Lay Summary (English)',
       'Lay Summary  Lead (German)', 'Lay Summary (German)',
       'Lay Summary Lead (French)', 'Lay Summary (French)',
       'Lay Summary Lead (Italian)', 'Lay Summary (Italian)'],
      dtype='object')

In [5]:
len([x for x in snsf_ge_abs_df["Abstract"] if str(x) == "nan"]) /  len(snsf_ge_abs_df["Abstract"])

0.5877609108159393

In [6]:
len([x for x in snsf_ge_abs_df["Project Title English"] if str(x) == "nan"]) /  len(snsf_ge_abs_df["Project Title English"])

0.5761253426101623

In [7]:
len([x for x in snsf_ge_abs_df["Project Title"] if str(x) == "nan"]) /  len(snsf_ge_abs_df["Project Title"])

0.0

In [8]:
abstracts = [x for x in snsf_ge_abs_df["Abstract"] if str(x) != "nan"]

In [9]:
len(set(snsf_ge_abs_df["Discipline Name"]))

159

In [10]:
set(snsf_ge_abs_df["Discipline Name"]);

In [11]:
snsf_ge_abs_df["Start Date"][0].split(".")[2]

'1993'

In [44]:
years_15_18 = [x for x in snsf_ge_abs_df["End Date"] if str(x) != 'nan' and 2015 <= int(str(x).split(".")[2]) <= 2018]
years_15_19 = [x for x in snsf_ge_abs_df["End Date"] if str(x) != 'nan' and 2015 <= int(str(x).split(".")[2]) <= 2019]
years_15_18_ind = [i for i,x in enumerate(snsf_ge_abs_df["End Date"]) if str(x) != 'nan' and 2015 <= int(str(x).split(".")[2]) <= 2018]
years_15_19_ind = [i for i,x in enumerate(snsf_ge_abs_df["End Date"]) if str(x) != 'nan' and 2015 <= int(str(x).split(".")[2]) <= 2019]

In [45]:
len(years_15_18)

10857

In [14]:
English = snsf_ge_abs_df["Project Title"][[i for i,x in enumerate(snsf_ge_abs_df["Project Title English"])]]

In [15]:
English[1:10]

1    A safe haven for untold casualties? An assessm...
2    Die übelste aller Frauen? Klytaimestra in Text...
3    Functional analysis of novel VEGF receptor 2 l...
4    'kuhghyer'. Eine historisch-anthropologische U...
5                   Méthodologie économique et espace.
6    Physically-Based Modeling for Point-Based Grap...
7    Le Journal helvétique (1732-1784): un espace d...
8    CollAGEing:  understanding age- and diabetes-r...
9    Vorschläge für eine Bauzonenumschreibung im Bu...
Name: Project Title, dtype: object

In [16]:
len([x for x in snsf_ge_abs_df["Funding Instrument"] if str(x) != "nan"])/len(snsf_ge_abs_df)

1.0

In [17]:
snsf_ge_abs_df["Funding Instrument Hierarchy"][1:50]

1                                   Careers;Fellowships
2                                 Science communication
3     Careers;Academic and research careers (non-pro...
4                                       Project funding
5                                   Careers;Fellowships
6                                       Project funding
7                                            Programmes
8                                               Careers
9        Programmes;National Research Programmes (NRPs)
10                                  Careers;Fellowships
11                                  Careers;Fellowships
12       Programmes;National Research Programmes (NRPs)
13                                Science communication
14                                              Careers
15                                  Careers;Fellowships
16                                      Project funding
17                                Science communication
18                                              

In [18]:
snsf_ge_abs_df["Funding Instrument"]

0                   Project funding (Div. I-III)
1                         Early Postdoc.Mobility
2                             Publication grants
3                     Marie Heim-Voegtlin grants
4                      Project funding (special)
                          ...                   
75883                               FORCE / CERN
75884    Fellowships for prospective researchers
75885               Project funding (Div. I-III)
75886    Fellowships for prospective researchers
75887               Project funding (Div. I-III)
Name: Funding Instrument, Length: 75888, dtype: object

In [19]:
snsf_ge_abs_df["Institution"]

0        Département de Physiologie Faculté de Biologie...
1        Pelagic Ecosystems Research Group  Hawaii Paci...
2        Seminar für Griechische und Lateinische Philol...
3                                   Paul Scherrer Institut
4                        Kantonsbibliothek Appenzell A.Rh.
                               ...                        
75883                 Departement Physik Universität Basel
75884    UNI: Princeton University Dept of Geological &...
75885    Zoologisches Institut Bio-/Pharmazentrum Unive...
75886         UNI: Unviersity of California   La Jolla USA
75887    Institut de Police Scientifique Ecole des Scie...
Name: Institution, Length: 75888, dtype: object

In [20]:
snsf_ge_abs_df["University"]

0          University of Lausanne - LA
1            Institution abroad - IACH
2            University of Zurich - ZH
3        Paul Scherrer Institute - PSI
4            University of Zurich - ZH
                     ...              
75883         University of Basel - BS
75884        Institution abroad - IACH
75885         University of Basel - BS
75886        Institution abroad - IACH
75887      University of Lausanne - LA
Name: University, Length: 75888, dtype: object

In [21]:
snsf_ge_abs_df["Project Number String"]

0          3100-037668
1        P2SKP3_184044
2        B-0010-142744
3        PMCDP3_147530
4          1115-055486
             ...      
75883      2166-053297
75884      81NE-020428
75885      3100-008677
75886      81BS-068472
75887    105211_188898
Name: Project Number String, Length: 75888, dtype: object

In [22]:
len(snsf_ge_abs_df)

75888

In [23]:
len([x for x in snsf_ge_abs_df.iloc[years_15_18_ind,:]["Lay Summary (Italian)"] if str(x) != "nan"])

649

In [24]:
snsf_ge_abs_df.iloc[years_15_18_ind,:]

,Project Number,Project Number String,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,Institution Country,University,...,Keywords,Abstract,Lay Summary Lead (English),Lay Summary (English),Lay Summary Lead (German),Lay Summary (German),Lay Summary Lead (French),Lay Summary (French),Lay Summary Lead (Italian),Lay Summary (Italian)
17,166723,10CO18_166723,2016 International Conference on Technologies ...,NaN,Bolay Jean-Claude,Scientific Conferences,Science communication,VPAI - Coopération EPFL,Switzerland,EPF Lausanne - EPFL,...,Development Engineering; Sustainable Developme...,The Cooperation and Development Center (CODEV)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,143969,200021_143969,Ultradense silicon nanowires,Ultradense silicon nanowires,Ekinci Yasin,Project funding (Div. I-III),Project funding,Labor für Mikro- und Nanotechnologie Paul Sche...,Switzerland,Paul Scherrer Institute - PSI,...,nanowire; nanomanufacturing; silicon; material...,"Progress in micro and nanoelectronics, as pred...",NaN,"<p>Progress in micro and nanoelectronics, as p...",NaN,<p>Der rasante Fortschritt in der Mikro-und Na...,NaN,NaN,NaN,NaN
35,163509,B-OA10_163509,L’invention des Carpates: géographies d’une co...,NaN,Verlag Peter Lang AG Europäischer Verlag der W...,OAPEN-CH pilot project,Science communication,Verlag Peter Lang AG Europäischer Verlag der W...,Switzerland,University of Geneva - GE,...,Construction régionale transnationale ; Gestio...,La signature le 22 mai 2003 à Kiev de la Conve...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,182925,10BP12_182925,Unter Kriegsrecht. Die schweizerische Militärj...,Under Laws of War: Swiss Military Justice 1914...,Steiner Sebastian,Open Access Books,Science communication,Abt. Neueste Geschichte u. Zeitgeschichte Hist...,Switzerland,University of Berne - BE,...,Erster Weltkrieg; Schweiz; Militärjustiz; Land...,Die vorliegende Dissertation untersucht die sc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,154717,PZ00P3_154717,Characterizing migrations with modern and anci...,NaN,Malaspinas Anna Sapfo,Ambizione,Careers,Institut für Ökologie und Evolution Universitä...,Switzerland,University of Berne - BE,...,molecular evolution; Americas; population geno...,BACKGROUND: The mode and tempo of the coloniza...,NaN,NaN,NaN,NaN,NaN,<p><strong>Titre</strong></p><p>Caract&eacute;...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75849,189521,10BP12_189521,Des réseaux et des hommes. Participation et co...,Networks and Men. Swiss Participation and Cont...,Quin Grégory,Open Access Books,Science communication,PHASE Institut des sciences du sport/SSP Unive...,Switzerland,University of Lausanne - LA,...,Histoire du sport; Histoire suisse; Réseaux; B...,"Depuis les dernières années du 19ème siècle, d...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75852,189691,IZSEZ0_189691,Colloque International Clinique du sens,NaN,Brandt Pierre-Yves,Scientific Exchanges,Science communication,Institut de sciences sociales des religions IS...,Switzerland,University of Lausanne - LA,...,religion/spiritualité et santé; spiritual care...,"Certaines personnalités très médiatisées, tell...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75855,163447,31003A_163447,Post-transcriptional gene regulation in develo...,Post-transcriptional gene regulation in develo...,Grosshans Helge,Project funding (Div. I-III),Project funding,Friedrich Miescher Institute for Biomedical Re...,Switzerland,Institute Friedrich Miescher - FMI,...,translational control; developmental timing; L...,Faithful development of an organism requires p...,NaN,NaN,NaN,<p>Wir haben k&uuml;rzlich im Fadenwurm <em>Ca...,NaN,NaN,NaN,NaN
75871,172702,200020_172702,The build-up to volcanic eruptions,NaN,Caricchi Luca,Project funding (Div. I-III),Project funding,Département des sciences de la Terre Universit...,Switzerland,University of Geneva - GE,...,Plutons; zircon geochronology; cross correlati...,I seek funding to comp

In [27]:

snsf_collab = pd.read_csv("../data/P3_CollaborationExport.csv", sep=";")
snsf_collab.keys()

Index(['Project Number', 'Group/Person', 'Types of collaboration', 'Country',
       'Project Start Date', 'Project End Date'],
      dtype='object')

In [33]:
snsf_collab.head()

,Project Number,Group/Person,Types of collaboration,Country,Project Start Date,Project End Date,years
0,67355,Office for History of Science and Technology U...,"in-depth/constructive exchanges on approaches,...",United States of America,01.01.2002,30.04.2003,2002
1,100150,"Colorado Division of Wildlife, Fort Collins",Publication,United States of America,01.04.2003,31.07.2007,2003
2,100851,Harvard Divinity School,"in-depth/constructive exchanges on approaches,...",United States of America,01.08.2003,31.07.2004,2003
3,103945,Functional Genomics Center Zurich,"in-depth/constructive exchanges on approaches,...",Switzerland,01.08.2004,31.01.2009,2004
4,106366,Universitäten Kiel und Heidelberg (Ludger Lieb),"in-depth/constructive exchanges on approaches,...",Germany,01.10.2004,30.09.2005,2004


In [29]:
snsf_collab["years"] = [int(str(x).split(".")[2]) for x in snsf_collab["Project Start Date"]]
snsf_collab["years"]

0        2002
1        2003
2        2003
3        2004
4        2004
         ... 
63832    2021
63833    2021
63834    2021
63835    2021
63836    2021
Name: years, Length: 63837, dtype: int64

In [60]:
collab_15_18 = pd.DataFrame([snsf_collab.iloc[i,:] for i in range(len(snsf_collab)) if 2015 <= snsf_collab["years"][i] <= 2018])
collab_15_19 = pd.DataFrame([snsf_collab.iloc[i,:] for i in range(len(snsf_collab)) if 2015 <= snsf_collab["years"][i] <= 2019])
collab_15_18["Project Number"] = [int(i) for i in collab_15_18["Project Number"]]
collab_15_19["Project Number"] = [int(i) for i in collab_15_19["Project Number"]]
print(len(set(collab_15_18["Project Number"])),  len(set(collab_15_19["Project Number"])))

5874 6659


In [48]:
len(set(collab_15_18["Project Number"])) / len(years_15_18)

0.541033434650456

In [32]:
len([snsf_collab.iloc[i,:] for i in range(len(snsf_collab)) if 2015 <= snsf_collab["years"][i] <= 2019])

30194

In [55]:

years_15_18_df = pd.DataFrame([snsf_ge_abs_df.iloc[i,:] for i,x in enumerate(snsf_ge_abs_df["End Date"]) if str(x) != 'nan' and 2015 <= int(str(x).split(".")[2]) <= 2018])
len(years_15_18_df)

10857

In [56]:
years_15_18_df.keys()

Index(['Project Number', 'Project Number String', 'Project Title',
       'Project Title English', 'Responsible Applicant', 'Funding Instrument',
       'Funding Instrument Hierarchy', 'Institution', 'Institution Country',
       'University', 'Discipline Number', 'Discipline Name',
       'Discipline Name Hierarchy', 'All disciplines', 'Start Date',
       'End Date', 'Approved Amount', 'Keywords', 'Abstract',
       'Lay Summary Lead (English)', 'Lay Summary (English)',
       'Lay Summary  Lead (German)', 'Lay Summary (German)',
       'Lay Summary Lead (French)', 'Lay Summary (French)',
       'Lay Summary Lead (Italian)', 'Lay Summary (Italian)'],
      dtype='object')

In [58]:
# importing the module
import collections

# using Counter to find frequency of elements
freq_rd = collections.Counter(years_15_18_df["Discipline Name"])
freq_rd.most_common()

[('General history (without pre-and early history)', 408),
 ('Psychology', 346),
 ('Molecular Biology', 329),
 ('Neurophysiology and Brain Research', 319),
 ('Mathematics', 308),
 ('Condensed Matter Physics', 303),
 ('Information Technology', 292),
 ('Legal sciences', 290),
 ('Sociology', 288),
 ('Economics', 275),
 ('German and English languages and literature', 254),
 ('Political science', 240),
 ('Physical Chemistry', 234),
 ('Immunology, Immunopathology', 221),
 ('Material Sciences', 216),
 ('Philosophy', 213),
 ('Organic Chemistry', 205),
 ('Ecology', 197),
 ('Romance languages and literature', 188),
 ('Education and learning sciences, subject-specific education', 183),
 ('Cellular Biology, Cytology', 177),
 ('Science of management', 165),
 ('Religious studies, Theology', 164),
 ('Visual arts and Art history', 157),
 ('Experimental Cancer Research', 155),
 ('Other disciplines of Environmental Sciences', 154),
 ('Astronomy, Astrophysics and Space Sciences', 152),
 ('Biochemistry', 

In [61]:

freq_rd = collections.Counter(years_15_18_df["Institution Country"])

In [62]:
freq_rd

Counter({'Switzerland': 7680,
         'United States of America': 1083,
         'Great Britain and Northern Ireland': 398,
         'Australia': 76,
         nan: 724,
         'Norway': 11,
         'Canada': 126,
         'Germany': 225,
         'Poland': 2,
         'France': 132,
         'Spain': 35,
         'Israel': 15,
         'Belgium': 45,
         'Brazil': 4,
         'Japan': 12,
         'Netherlands': 69,
         'Sweden': 40,
         'Italy': 36,
         'South Africa': 12,
         'Finland': 14,
         'Peru': 3,
         'Austria': 29,
         'Denmark': 15,
         'Czech Republic': 1,
         'Kenya': 3,
         'Malta': 1,
         'Morocco': 2,
         'Ireland': 6,
         'Taiwan': 1,
         'Russia': 12,
         'Mexico': 7,
         'Greece': 4,
         'Panama': 1,
         'New Zealand': 6,
         'Hongkong': 1,
         'Korean Republic (South Korea)': 1,
         'Singapore': 3,
         'China': 4,
         'Portugal': 8,
         '

In [64]:

a = collections.Counter(years_15_18_df["Discipline Name Hierarchy"])

In [65]:
len(a)

21

In [66]:
a

Counter({'Humanities and Social Sciences;Economics, law': 730,
         'Mathematics, Natural- and Engineering Sciences;Physics': 675,
         'Humanities and Social Sciences;Ethnology, social and human geography': 235,
         'Humanities and Social Sciences;Theology & religious studies, history, classical studies, archaeology, prehistory and early history': 874,
         'Biology and Medicine;Clinical Medicine': 419,
         'Mathematics, Natural- and Engineering Sciences;Chemistry': 564,
         'Humanities and Social Sciences;Psychology, educational studies': 547,
         'Biology and Medicine;General Biology': 414,
         'Biology and Medicine;Basic Biological Research': 1060,
         'Biology and Medicine;Basic Medical Sciences': 591,
         'Mathematics, Natural- and Engineering Sciences': 460,
         'Mathematics, Natural- and Engineering Sciences;Engineering Sciences': 1033,
         'Humanities and Social Sciences;Linguistics and literature, philosophy': 830,
    

In [67]:
len(set(snsf_ge_abs_df["Project Number"]))


75888

In [68]:


len(snsf_ge_abs_df["Project Number"])

75888